# Geometry Optimization

We present here a complete example on how to use QMCTorch to optimize the geometry of a H2 molecule.
As previously the firs task is to import all the modules needed

In [1]:
from torch import optim
from torch.optim import Adam
from qmctorch.wavefunction import SlaterJastrow
from qmctorch.solver import SolverSlaterJastrow
from qmctorch.sampler import Metropolis
from qmctorch.scf import Molecule
from qmctorch.utils import plot_energy

INFO:QMCTorch|  ____    __  ______________             _
INFO:QMCTorch| / __ \  /  |/  / ___/_  __/__  ________/ /  
INFO:QMCTorch|/ /_/ / / /|_/ / /__  / / / _ \/ __/ __/ _ \ 
INFO:QMCTorch|\___\_\/_/  /_/\___/ /_/  \___/_/  \__/_//_/ 


We then define the molecule. We create a H2 molecule with a short atomic distance that we will then try to relax


In [2]:
mol = Molecule(atom = 'H 0. 0. 0.25; H 0. 0. -0.25', unit='bohr', calculator='pyscf', basis='sto-3g', redo_scf=True)

INFO:QMCTorch|
INFO:QMCTorch| SCF Calculation
INFO:QMCTorch|  Removing H2_pyscf_sto-3g.hdf5 and redo SCF calculations
INFO:QMCTorch|  Running scf  calculation
converged SCF energy = -0.40332644165448
INFO:QMCTorch|  Molecule name       : H2
INFO:QMCTorch|  Number of electrons : 2
INFO:QMCTorch|  SCF calculator      : pyscf
INFO:QMCTorch|  Basis set           : sto-3g
INFO:QMCTorch|  SCF                 : HF
INFO:QMCTorch|  Number of AOs       : 2
INFO:QMCTorch|  Number of MOs       : 2
INFO:QMCTorch|  SCF Energy          : -0.403 Hartree


As for the wave function optimization example we need to define a wave function, a sampler an optimizer and a solver.

In [3]:
# wave function with only the ground state determinant
wf = SlaterJastrow(mol, configs='ground_state').gto2sto()

# sampler
sampler = Metropolis(nwalkers=200, nstep=2000, step_size=0.5,
                     nelec=wf.nelec, ndim=wf.ndim,
                     init=mol.domain('normal'),
                     move={'type': 'all-elec', 'proba': 'normal'})
# optimizer
opt = Adam(wf.parameters(), lr=0.005)

# scheduler
scheduler = optim.lr_scheduler.StepLR(opt, step_size=20, gamma=0.75)

INFO:QMCTorch|
INFO:QMCTorch| Wave Function
INFO:QMCTorch|  Jastrow factor      : True
INFO:QMCTorch|  Jastrow kernel      : PadeJastrowKernel
INFO:QMCTorch|  Highest MO included : 2
INFO:QMCTorch|  Configurations      : ground_state
INFO:QMCTorch|  Number of confs     : 1
INFO:QMCTorch|  Kinetic energy      : jacobi
INFO:QMCTorch|  Number var  param   : 18
INFO:QMCTorch|  Cuda support        : False
INFO:QMCTorch|  Fit GTOs to STOs  : 
INFO:QMCTorch|
INFO:QMCTorch| Wave Function
INFO:QMCTorch|  Jastrow factor      : True
INFO:QMCTorch|  Jastrow kernel      : PadeJastrowKernel
INFO:QMCTorch|  Highest MO included : 2
INFO:QMCTorch|  Configurations      : ground_state
INFO:QMCTorch|  Number of confs     : 1
INFO:QMCTorch|  Kinetic energy      : jacobi
INFO:QMCTorch|  Number var  param   : 14
INFO:QMCTorch|  Cuda support        : False
INFO:QMCTorch|
INFO:QMCTorch| Monte-Carlo Sampler
INFO:QMCTorch|  Number of walkers   : 1000
INFO:QMCTorch|  Number of steps     : 2000
INFO:QMCTorch|  Ste

In [5]:
solver = SolverSlaterJastrow(wf=wf,
                       sampler=sampler,
                       optimizer=opt,
                       scheduler=scheduler)
solver.configure(loss='energy', grad='manual', track=['local_energy','geometry'])

INFO:QMCTorch|
INFO:QMCTorch| Warning : dump to hdf5
INFO:QMCTorch| Object SolverSlaterJastrow already exists in H2_pyscf_sto-3g_QMCTorch.hdf5
INFO:QMCTorch| Object name changed to SolverSlaterJastrow_2
INFO:QMCTorch|
INFO:QMCTorch|
INFO:QMCTorch| QMC Solver 
INFO:QMCTorch|  WaveFunction        : SlaterJastrow
INFO:QMCTorch|  Sampler             : Metropolis
INFO:QMCTorch|  Optimizer           : Adam
INFO:QMCTorch|  Scheduler           : StepLR


To optimize the geometry of the molecule we must use the `set_params_requires_grad` method of the solver. We here set all the wave function parameters and all the atomic positions as variational parameters, leading to a simultaneous optimization of the wave function and of the atomic coordinates. We can then run the optimization here using 50 epochs

In [6]:
solver.set_params_requires_grad(wf_params=True, geo_params=True)
obs = solver.run(50)


RuntimeError: expected scalar type Double but found Float

In [10]:
solver.save_traj('h2_traj.xyz', obs)

IndexError: list index out of range

In [13]:
obs.geometry

[]